In [5]:
import requests
import numpy as np
import pandas as pd
import regex as re
import sklearn
import gensim
import pickle
from utils import *
from processor import PreProcessor


saveFileName = "data/dialects.csv"
isDownload = False

In [6]:
dialects_df = readCSV("data/dialect_dataset.csv")
dialects_arr = np.array(dialects_df,dtype="str")

ids = dialects_arr[:,0]
dialects = dialects_arr[:,1]
print(dialects_arr.shape)

(458197, 2)


In [7]:
if isDownload:
    tweets = get_tweets(ids)
    print(tweets.shape)
    
    dialects_data = np.array([ids,tweets,dialects]).T
    saveCSV(dialects_data,saveFileName,True)
else:    
    dialects_data = readCSV(saveFileName,True,"str")
    
np.random.shuffle(dialects_data)
print(dialects_data.shape)

ids,tweets,dialects = dialects_data[:,0],dialects_data[:,1],dialects_data[:,2]
dataSize = dialects_data.shape[0]

(458197, 3)


In [9]:
preprocessor = PreProcessor(w2vPath = "weights/full_grams_cbow_100_twitter.mdl",isRemove = True)
encodedDialects,no_classes,labels2idx,idx2labels = labelsEncoder(dialects)
oneHotDialects = oneHotEncoder(encodedDialects,no_classes)
print(oneHotDialects.shape)

idx = np.random.randint(0,ids.shape[0])
print(f"id : {ids[idx]}\ntweet : {tweets[idx]}\ndialect : {dialects[idx]}")

Loading Word2vec
(458197, 18)
id : 1156375592847233024
tweet : @NadaAkkila1 لا ندى ، اكيد فرق كبير ، بس هالشخص نفسيته حلوه ، اسم الله ومتاكده انه مظلوم ماديا
dialect : PL


In [10]:
print(preprocessor.identifyMentions(tweets[idx]))
print(preprocessor.identifyMentions("@imesS0852_boy https:///gomeogme/fmofw @1998 email@gmail.com @ahmedmohamed \n @gn_494irnirhr"))

 @USER لا ندى ، اكيد فرق كبير ، بس هالشخص نفسيته حلوه ، اسم الله ومتاكده انه مظلوم ماديا
 @USER https:///gomeogme/fmofw  @USER email@gmail.com  @USER 
  @USER


In [11]:
print(preprocessor.identifyURL(tweets[idx]))
print(preprocessor.identifyURL("@imesS0852_boy https://www.google.com/in-go/123456:8080-98 @1998 email@gmail.com www.facebook.com @ahmedmohamed \n @gn_494irnirhr www.youtube.com/video=15020"))

@NadaAkkila1 لا ندى ، اكيد فرق كبير ، بس هالشخص نفسيته حلوه ، اسم الله ومتاكده انه مظلوم ماديا
@imesS0852_boy  @URL  @1998 email@gmail.com  @URL  @ahmedmohamed 
 @gn_494irnirhr  @URL 


In [12]:
print(preprocessor.identifyHashTag(tweets[idx]))
print(preprocessor.identifyHashTag("#hash1_2 @imesS0852_boy https://www.google.com/in/123456:8080-98 #hash2_3 @1998 email@gmail.com www.facebook.com @ahmedmohamed \n @gn_494irnirhr #hash4_5"))

@NadaAkkila1 لا ندى ، اكيد فرق كبير ، بس هالشخص نفسيته حلوه ، اسم الله ومتاكده انه مظلوم ماديا
 @HASH @imesS0852_boy https://www.google.com/in/123456:8080-98  @HASH @1998 email@gmail.com www.facebook.com @ahmedmohamed 
 @gn_494irnirhr  @HASH 


In [13]:
print(preprocessor.removePunctuations(tweets[idx]))
print(preprocessor.removePunctuations("'How are you doing ?!',\nsays mohamed."))

@NadaAkkila1 لا ندى  اكيد فرق كبير  بس هالشخص نفسيته حلوه  اسم الله ومتاكده انه مظلوم ماديا
How are you doing 
says mohamed


In [14]:
print(preprocessor.identifyEmojis(tweets[idx]))
print(preprocessor.identifyEmojis(u'This is a smiley face \U0001f602'))

@NadaAkkila1 لا ندى ، اكيد فرق كبير ، بس هالشخص نفسيته حلوه ، اسم الله ومتاكده انه مظلوم ماديا
This is a smiley face  @EMOJI 


In [15]:
print("before cleaning : ", tweets[idx+3])
print("after cleaning : ",preprocessor.cleanTweet(tweets[idx+3].lower()))
print(preprocessor.cleanTweet("@mohamedessam98_20  this is a cleaned tweet \U0001f602 ! @AIM ! from wwww.twitter.com. #CLEANTWEET"))

before cleaning :  @AllTawfeeq
كيف يعني

غصب يرشحوا عربي
after cleaning :  @USER كيف يعني غصب يرشحوا عربي
@USER this is a cleaned tweet @EMOJI @USER from @URL @HASH


In [16]:
cleanedTweets = preprocessor.cleanTweets(tweets,isRemove=True)
preprocessor.buildVocabulary(cleanedTweets,True)

In [17]:
tweetsTokens,masks = preprocessor.tokenizeTweets(cleanedTweets)
print(tweetsTokens.shape)

(458197, 70)


In [18]:
preTrainedEmbed = preprocessor.loadPreTrainedEmbeddings()
print(preTrainedEmbed.shape)

(446654, 100)


In [19]:
sentVecs = preprocessor.sentToVec(tweetsTokens,preTrainedEmbed)
print(sentVecs.shape)

(458197, 100)


In [20]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
sentVecsRed = pca.fit_transform(sentVecs)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)
print(sentVecsRed.shape)

[0.33694774 0.0798965  0.06166664 0.04466995 0.03656096 0.02677004
 0.02385762 0.02223515 0.018316   0.01553069 0.01343988 0.01264321
 0.01183316 0.01090182 0.01071412 0.00997094 0.00912391 0.0085606
 0.00814308 0.00741792]
[206.15690655 100.38769591  88.19455392  75.06272651  67.90870245
  58.10869219  54.85676842  52.95862127  48.06533177  44.26005919
  41.17316807  39.93422854  38.63375687  37.08226213  36.76164145
  35.46375854  33.92400978  32.86008891  32.04873886  30.58847492]
(458197, 20)


In [21]:
X_train,X_test,Y_train,Y_test,masks_train,masks_test = splitData(sentVecsRed,dialects,masks)

In [23]:
from sklearn.svm import LinearSVC
a
model = LinearSVC()

samples = len(Y_train)

model,results = train_predict(model, samples, X_train, Y_train, X_test, Y_test)

pickle.dump(model,open('models/svm.pkl',"wb"))
pickle.dump(pca,open('models/pca.pkl',"wb"))

print(results)

LinearSVC trained on 320737 samples.
{'train_time': 77.45333957672119, 'pred_time': 0.411144495010376, 'acc_train': 0.2718426623682332, 'acc_test': 0.2732358504292158}
